# RTW MAM Model V1 
# County Data First

In [91]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import csv
import pickle

In [84]:
#SET DATE FILTERS

prevalence = ['2020-06-15','2020-06-14','2020-06-13','2020-06-12','2020-06-11','2020-06-10','2020-06-09','2020-06-08',
              '2020-06-07','2020-06-06','2020-06-05','2020-06-04','2020-06-03','2020-06-02']

day7_current = ['2020-06-15','2020-06-14','2020-06-13','2020-06-12','2020-06-11','2020-06-10','2020-06-09']

day7_previous = ['2020-06-08','2020-06-07','2020-06-06','2020-06-05','2020-06-04','2020-06-03','2020-06-02']

In [107]:
#Creating pop file
#df = pd.read_csv("County4techs_4_15.csv",names=["I", "day", "cases", "county","state","doubling","fips","pop"],
#                dtype={'fips': str})

#df_filtered = df[(df.day==5)]
#print(df.shape)

#pop_df = df_filtered[['fips','county','state','pop' ]]
#print(pop_df.head())

#pickle.dump(pop_df, open("pop_df.sav","wb" ))

    fips       county       state         pop
1  53061    Snohomish  Washington    822083.0
3  17031         Cook    Illinois   5150233.0
5  06059       Orange  California   3175692.0
7  04013     Maricopa     Arizona   4485414.0
9  06037  Los Angeles  California  10039107.0


In [108]:
#Bring in County Data set
nytimes = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
counties = pd.read_csv(nytimes,dtype={'fips': str})
print(nyt.head())

#Change to own pop_df location
pop_df = pickle.load(open("pop_df.sav","rb"))

print(pop_df.head())
print(type(pop_df))

         date     county       state   fips  cases  deaths
0  2020-01-21  Snohomish  Washington  53061      1       0
1  2020-01-22  Snohomish  Washington  53061      1       0
2  2020-01-23  Snohomish  Washington  53061      1       0
3  2020-01-24       Cook    Illinois  17031      1       0
4  2020-01-24  Snohomish  Washington  53061      1       0
    fips       county       state         pop
1  53061    Snohomish  Washington    822083.0
3  17031         Cook    Illinois   5150233.0
5  06059       Orange  California   3175692.0
7  04013     Maricopa     Arizona   4485414.0
9  06037  Los Angeles  California  10039107.0
<class 'pandas.core.frame.DataFrame'>


# Create Active Case Count

In [115]:
#Create new cases field
#Sort df by Fips and Date
counties.sort_values(by=['fips','date'],inplace=True, ascending=True)
print(counties.head())

#Take difference between rows for new case and new death numbers
counties['case_diff'] = counties.cases.diff()
counties['death_diff'] = counties.deaths.diff()


print(counties.head())

             date   county    state   fips  cases  deaths
9480   2020-03-24  Autauga  Alabama  01001      1       0
10832  2020-03-25  Autauga  Alabama  01001      4       0
12361  2020-03-26  Autauga  Alabama  01001      6       0
14021  2020-03-27  Autauga  Alabama  01001      6       0
15801  2020-03-28  Autauga  Alabama  01001      6       0
             date   county    state   fips  cases  deaths  case_diff  \
9480   2020-03-24  Autauga  Alabama  01001      1       0        NaN   
10832  2020-03-25  Autauga  Alabama  01001      4       0        3.0   
12361  2020-03-26  Autauga  Alabama  01001      6       0        2.0   
14021  2020-03-27  Autauga  Alabama  01001      6       0        0.0   
15801  2020-03-28  Autauga  Alabama  01001      6       0        0.0   

       death_diff  
9480          NaN  
10832         0.0  
12361         0.0  
14021         0.0  
15801         0.0  


In [38]:
#Filter for past 14 days
range_14 = counties[counties['date'].isin(prevalence)]
print(range_14.head(15))

              date   county    state   fips  cases  deaths  case_diff  \
197593  2020-06-02  Autauga  Alabama  01001    240       5        6.0   
200589  2020-06-03  Autauga  Alabama  01001    240       5        0.0   
203588  2020-06-04  Autauga  Alabama  01001    242       5        2.0   
206590  2020-06-05  Autauga  Alabama  01001    249       5        7.0   
209596  2020-06-06  Autauga  Alabama  01001    260       5       11.0   
212603  2020-06-07  Autauga  Alabama  01001    266       5        6.0   
215613  2020-06-08  Autauga  Alabama  01001    273       5        7.0   
218625  2020-06-09  Autauga  Alabama  01001    283       5       10.0   
221641  2020-06-10  Autauga  Alabama  01001    296       6       13.0   
224663  2020-06-11  Autauga  Alabama  01001    316       6       20.0   
227686  2020-06-12  Autauga  Alabama  01001    327       6       11.0   
230711  2020-06-13  Autauga  Alabama  01001    335       6        8.0   
233737  2020-06-14  Autauga  Alabama  01001    361 

In [39]:
#Group by FIPS, sum up new Cases for prevalence - for 7 day averages will take mean after filtering for 7 days
prevalence_grouped = range_14.groupby(['fips'], as_index=False).sum()

print(prevalence_grouped.head(25))

     fips  cases  deaths  case_diff  death_diff
0   01001   4056      76      134.0         1.0
1   01003   4771     126       77.0         0.0
2   01005   2845      14       65.0         0.0
3   01007   1255      14       32.0         0.0
4   01009   1158      14       51.0         0.0
5   01011   3466     111       97.0         3.0
6   01013   6578     316      128.0         7.0
7   01015   2621      42       32.0         0.0
8   01017   5743     364      105.0         1.0
9   01019    617      56       14.0         2.0
10  01021   1700      24       34.0         1.0
11  01023   2252     146       22.0         1.0
12  01025   2296      36       45.0         1.0
13  01027    409      28        2.0         0.0
14  01029    253      14        7.0         0.0
15  01031   3613      14       53.0         0.0
16  01033   3139      55       83.0         3.0
17  01035    871      14       61.0         0.0
18  01037    584      14        6.0         0.0
19  01039   1493      14       90.0     

In [109]:
#Join in Prevalence to 
prevalence_w_pop = pd.merge(pop_df,
                 prevalence_grouped[['fips','case_diff']],
                 on='fips', 
                how='left')
print(prevalence_w_pop.head(10))

    fips         county          state         pop  case_diff
0  53061      Snohomish     Washington    822083.0      216.0
1  17031           Cook       Illinois   5150233.0     6689.0
2  06059         Orange     California   3175692.0     2227.0
3  04013       Maricopa        Arizona   4485414.0     9435.0
4  06037    Los Angeles     California  10039107.0    17823.0
5  06085    Santa Clara     California   1927852.0      425.0
6  25025        Suffolk  Massachusetts    803907.0      753.0
7  06075  San Francisco     California    881549.0      382.0
8  55025           Dane      Wisconsin    546695.0      231.0
9  06073      San Diego     California   3338330.0     2056.0


In [110]:
prevalence_w_pop['active_cases_100k'] = ((prevalence_w_pop['case_diff']/prevalence_w_pop['pop'])*100000)
print(prevalence_w_pop.head())

    fips       county       state         pop  case_diff  active_cases_100k
0  53061    Snohomish  Washington    822083.0      216.0          26.274719
1  17031         Cook    Illinois   5150233.0     6689.0         129.877619
2  06059       Orange  California   3175692.0     2227.0          70.126448
3  04013     Maricopa     Arizona   4485414.0     9435.0         210.348476
4  06037  Los Angeles  California  10039107.0    17823.0         177.535711


# Create 7 Day Rolling Avgs for Cases & Deaths

In [116]:
#Filter Diff datasets for current and previous 7 day periods

#join prev and current together with pop

range_current7 = counties[counties['date'].isin(day7_current)]
range_prev7 = counties[counties['date'].isin(day7_previous)]

print(range_current7.head())

              date   county    state   fips  cases  deaths  case_diff  \
218625  2020-06-09  Autauga  Alabama  01001    283       5       10.0   
221641  2020-06-10  Autauga  Alabama  01001    296       6       13.0   
224663  2020-06-11  Autauga  Alabama  01001    316       6       20.0   
227686  2020-06-12  Autauga  Alabama  01001    327       6       11.0   
230711  2020-06-13  Autauga  Alabama  01001    335       6        8.0   

        death_diff  
218625         0.0  
221641         1.0  
224663         0.0  
227686         0.0  
230711         0.0  


In [135]:
#Group by with mean
current_grouped = range_current7.groupby(['fips'],as_index=False).mean()
previous_grouped = range_prev7.groupby(['fips'], as_index=False).mean()

current_grouped['curr7_case'] = current_grouped['case_diff']
current_grouped['curr7_death'] = current_grouped['death_diff']
previous_grouped['prev7_case'] = previous_grouped['case_diff']
previous_grouped['prev7_death'] = previous_grouped['death_diff']
print(current_grouped.head())
print(previous_grouped.head())

    fips       cases    deaths  case_diff  death_diff  curr7_case  curr7_death
0  01001  326.571429  5.857143  13.571429    0.142857   13.571429     0.142857
1  01003  364.285714  9.000000   6.857143    0.000000    6.857143     0.000000
2  01005  220.857143  1.000000   5.714286    0.000000    5.714286     0.000000
3  01007   99.571429  1.000000   4.142857    0.000000    4.142857     0.000000
4  01009   96.571429  1.000000   5.857143    0.000000    5.857143     0.000000
    fips       cases  deaths  case_diff  death_diff  prev7_case  prev7_death
0  01001  252.857143     5.0   5.571429         0.0    5.571429          0.0
1  01003  317.285714     9.0   4.142857         0.0    4.142857          0.0
2  01005  185.571429     1.0   3.571429         0.0    3.571429          0.0
3  01007   79.714286     1.0   0.428571         0.0    0.428571          0.0
4  01009   68.857143     1.0   1.428571         0.0    1.428571          0.0


In [136]:
#Merge both sets with population
current_w_pop = pd.merge(pop_df,
                 current_grouped[['fips','curr7_case', 'curr7_death']],
                 on='fips', 
                how='left')
print(current_w_pop.head(10))

previous_w_pop = pd.merge(pop_df,
                 previous_grouped[['fips','prev7_case', 'prev7_death']],
                 on='fips', 
                how='left')
print(previous_w_pop.head(10))

    fips         county          state         pop   curr7_case  curr7_death
0  53061      Snohomish     Washington    822083.0    13.000000     0.428571
1  17031           Cook       Illinois   5150233.0   337.857143    35.285714
2  06059         Orange     California   3175692.0   159.428571     6.285714
3  04013       Maricopa        Arizona   4485414.0   767.000000     9.428571
4  06037    Los Angeles     California  10039107.0  1306.714286    38.714286
5  06085    Santa Clara     California   1927852.0    36.714286     0.857143
6  25025        Suffolk  Massachusetts    803907.0    43.714286     2.857143
7  06075  San Francisco     California    881549.0    24.714286     0.285714
8  55025           Dane      Wisconsin    546695.0    16.000000     0.142857
9  06073      San Diego     California   3338330.0   141.571429     1.142857
    fips         county          state         pop   prev7_case  prev7_death
0  53061      Snohomish     Washington    822083.0    17.857143     0.571429

In [137]:
current_w_pop['cur_7rollavg_cases'] = ((current_w_pop['curr7_case']/current_w_pop['pop'])*100000)
current_w_pop['cur_7rollavg_deaths'] = ((current_w_pop['curr7_death']/current_w_pop['pop'])*100000)

In [138]:
previous_w_pop['prev_7rollavg_cases'] = ((previous_w_pop['prev7_case']/previous_w_pop['pop'])*100000)
previous_w_pop['prev_7rollavg_deaths'] = ((previous_w_pop['prev7_death']/previous_w_pop['pop'])*100000)

In [139]:
#Merge with prevalence dataset
prev_curr = pd.merge(prevalence_w_pop,
                 current_w_pop[['fips','cur_7rollavg_cases', 'cur_7rollavg_deaths','curr7_case','curr7_death']],
                 on='fips', 
                how='left')
print(prev_curr.head(10))

    fips         county          state         pop  case_diff  \
0  53061      Snohomish     Washington    822083.0      216.0   
1  17031           Cook       Illinois   5150233.0     6689.0   
2  06059         Orange     California   3175692.0     2227.0   
3  04013       Maricopa        Arizona   4485414.0     9435.0   
4  06037    Los Angeles     California  10039107.0    17823.0   
5  06085    Santa Clara     California   1927852.0      425.0   
6  25025        Suffolk  Massachusetts    803907.0      753.0   
7  06075  San Francisco     California    881549.0      382.0   
8  55025           Dane      Wisconsin    546695.0      231.0   
9  06073      San Diego     California   3338330.0     2056.0   

   active_cases_100k  cur_7rollavg_cases  cur_7rollavg_deaths   curr7_case  \
0          26.274719            1.581349             0.052132    13.000000   
1         129.877619            6.560036             0.685129   337.857143   
2          70.126448            5.020278          

In [140]:
county_set = pd.merge(prev_curr,
                 previous_w_pop[['fips','prev_7rollavg_cases', 'prev_7rollavg_deaths','prev7_case','prev7_death']],
                 on='fips', 
                how='left')
print(county_set.head(10))

    fips         county          state         pop  case_diff  \
0  53061      Snohomish     Washington    822083.0      216.0   
1  17031           Cook       Illinois   5150233.0     6689.0   
2  06059         Orange     California   3175692.0     2227.0   
3  04013       Maricopa        Arizona   4485414.0     9435.0   
4  06037    Los Angeles     California  10039107.0    17823.0   
5  06085    Santa Clara     California   1927852.0      425.0   
6  25025        Suffolk  Massachusetts    803907.0      753.0   
7  06075  San Francisco     California    881549.0      382.0   
8  55025           Dane      Wisconsin    546695.0      231.0   
9  06073      San Diego     California   3338330.0     2056.0   

   active_cases_100k  cur_7rollavg_cases  cur_7rollavg_deaths   curr7_case  \
0          26.274719            1.581349             0.052132    13.000000   
1         129.877619            6.560036             0.685129   337.857143   
2          70.126448            5.020278          

In [141]:
#Create indicators for rolling averages
county_set['roll7case_diff'] = ((county_set['cur_7rollavg_cases']-county_set['prev_7rollavg_cases'])/county_set['prev_7rollavg_cases'])
county_set['roll7death_diff'] = ((county_set['cur_7rollavg_deaths']-county_set['prev_7rollavg_deaths'])/county_set['prev_7rollavg_deaths'])

print(county_set.head())

    fips       county       state         pop  case_diff  active_cases_100k  \
0  53061    Snohomish  Washington    822083.0      216.0          26.274719   
1  17031         Cook    Illinois   5150233.0     6689.0         129.877619   
2  06059       Orange  California   3175692.0     2227.0          70.126448   
3  04013     Maricopa     Arizona   4485414.0     9435.0         210.348476   
4  06037  Los Angeles  California  10039107.0    17823.0         177.535711   

   cur_7rollavg_cases  cur_7rollavg_deaths   curr7_case  curr7_death  \
0            1.581349             0.052132    13.000000     0.428571   
1            6.560036             0.685129   337.857143    35.285714   
2            5.020278             0.197932   159.428571     6.285714   
3           17.099871             0.210205   767.000000     9.428571   
4           13.016240             0.385635  1306.714286    38.714286   

   prev_7rollavg_cases  prev_7rollavg_deaths   prev7_case  prev7_death  \
0             2.17

In [142]:
#Write to CSV
county_set.to_csv('CountyMAMV2.csv') 